In [1]:
!pip install -U torch sahi

     ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
     ------------- -------------------------- 30.7/88.4 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 88.4/88.4 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/159.7 MB 3.3 MB/s eta 0:00:49
   ---------------------------------------- 0.2/159.7 MB 3.0 MB/s eta 0:00:53
   ---------------------------------------- 0.4/159.7 MB 2.9 MB/s eta 0:00:56
   ---------------------------------------- 0.6/159.7 MB 3.0 MB/s eta 0:00:53
   ---------------------------------------- 0.7/159.7 MB 3.2 MB/s eta 0:00:51
   ---------------------------------------- 0.9/159.7 MB 3.3 MB/s eta 0:00:48

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.0+cu121 requires torch==2.2.0+cu121, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.0+cu121 requires torch==2.2.0+cu121, but you have torch 2.3.0 which is incompatible.


In [9]:
! pip install torchvision

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
    --------------------------------------- 0.0/1.2 MB 330.3 kB/s eta 0:00:04
   - -------------------------------------- 0.0/1.2 MB 393.8 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.2 MB 901.1 kB/s eta 0:00:02
   -------- ------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.2 MB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 2.1 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.2 MB 2.3 MB/s eta 0:00:01
   -------------------------------- ------- 0.9/1.2 MB 2.6 MB/s eta 0:00:01
   -------------------------------------- - 1.1/1.2 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.7 MB/s eta 0:00:00
  Attempting unin

In [1]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.yolov8 import download_yolov8s_model

In [3]:
yolov8_model_path = f"../models/tuned/best-tuned.pt"
download_yolov8s_model(yolov8_model_path)
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8", model_path=yolov8_model_path, confidence_threshold=0.3
)


In [4]:
import sys
sys.path.append("../")
from utils import read_video, save_video


In [7]:
video_frames = read_video("../input_videos/08fd33_4.mp4")


In [8]:
result = get_sliced_prediction(video_frames[0], detection_model, 200, 200)

Performing prediction on 84 number of slices.


In [10]:
result.object_prediction_list

[ObjectPrediction<
     bbox: BoundingBox: <(1184.3223, 851.63837, 1203.5897, 867.16406), w: 19.2674560546875, h: 15.52569580078125>,
     mask: None,
     score: PredictionScore: <value: 0.4292224645614624>,
     category: Category: <id: 0, name: ball>>,
 ObjectPrediction<
     bbox: BoundingBox: <(1900.7249, 373.96344, 1920.0, 445.47583), w: 19.275146484375, h: 71.51239013671875>,
     mask: None,
     score: PredictionScore: <value: 0.4822448194026947>,
     category: Category: <id: 1, name: goalkeeper>>,
 ObjectPrediction<
     bbox: BoundingBox: <(1372.1463623046875, 816.5313720703125, 1444.3793115615845, 906.7301597595215), w: 72.23294925689697, h: 90.19878768920898>,
     mask: None,
     score: PredictionScore: <value: 0.9348105788230896>,
     category: Category: <id: 2, name: player>>,
 ObjectPrediction<
     bbox: BoundingBox: <(1308.046142578125, 445.87921142578125, 1354.3165283203125, 517.3377685546875), w: 46.2703857421875, h: 71.45855712890625>,
     mask: None,
     sco

In [14]:
import cv2
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.yolov8 import download_yolov8s_model
from pathlib import Path

from ultralytics.utils.files import increment_path

In [20]:

def run(weights="../models/tuned/best-tuned.pt", source="../examples/inputs/goal2.mp4", view_img=False, save_img=True, exist_ok=False):
    """
    Run object detection on a video using YOLOv8 and SAHI.

    Args:
        weights (str): Model weights path.
        source (str): Video file path.
        view_img (bool): Show results.
        save_img (bool): Save results.
        exist_ok (bool): Overwrite existing files.
    """

    # Check source path
    if not Path(source).exists():
        raise FileNotFoundError(f"Source path '{source}' does not exist.")

    yolov8_model_path = f"{weights}"
    download_yolov8s_model(yolov8_model_path)
    detection_model = AutoDetectionModel.from_pretrained(
        model_type="yolov8", model_path=yolov8_model_path, confidence_threshold=0.3, device="cuda:0"
    )

    # Video setup
    videocapture = cv2.VideoCapture(source)
    frame_width, frame_height = int(videocapture.get(3)), int(videocapture.get(4))
    fps, fourcc = int(videocapture.get(5)), cv2.VideoWriter_fourcc(*"mp4v")

    # Output setup
    save_dir = increment_path(Path("ultralytics_results_with_sahi") / "exp", exist_ok)
    save_dir.mkdir(parents=True, exist_ok=True)
    video_writer = cv2.VideoWriter(str(save_dir / f"{Path(source).stem}.mp4"), fourcc, fps, (frame_width, frame_height))

    while videocapture.isOpened():
        success, frame = videocapture.read()
        if not success:
            break

        results = get_sliced_prediction(
            frame, detection_model, slice_height=512, slice_width=512, overlap_height_ratio=0.2, overlap_width_ratio=0.2
        )
        object_prediction_list = results.object_prediction_list

        boxes_list = []
        clss_list = []
        for ind, _ in enumerate(object_prediction_list):
            boxes = (
                object_prediction_list[ind].bbox.minx,
                object_prediction_list[ind].bbox.miny,
                object_prediction_list[ind].bbox.maxx,
                object_prediction_list[ind].bbox.maxy,
            )
            clss = object_prediction_list[ind].category.name
            boxes_list.append(boxes)
            clss_list.append(clss)

        for box, cls in zip(boxes_list, clss_list):
            x1, y1, x2, y2 = box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (56, 56, 255), 2)
            label = str(cls)
            t_size = cv2.getTextSize(label, 0, fontScale=0.6, thickness=1)[0]
            cv2.rectangle(
                frame, (int(x1), int(y1) - t_size[1] - 3), (int(x1) + t_size[0], int(y1) + 3), (56, 56, 255), -1
            )
            cv2.putText(
                frame, label, (int(x1), int(y1) - 2), 0, 0.6, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA
            )

        if view_img:
            cv2.imshow(Path(source).stem, frame)
        if save_img:
            video_writer.write(frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    video_writer.release()
    videocapture.release()
    cv2.destroyAllWindows()

In [21]:
run()

Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing